In [27]:
import math, datetime, json, requests
import pandas as pd
import sklearn
from sklearn import svm, preprocessing
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [28]:

with open("config.json", "r") as configfile:
    config = json.load(configfile)

api_key = config["pf-api-key"]
data_resp = requests.get('https://perfectfall.com/api/ml-data/', headers={'token': api_key}).text
data_json = json.loads(data_resp)
df = pd.DataFrame(data_json)

df.drop_duplicates(subset=['game_id'], keep='last', inplace=True)

In [29]:
X = df[['neutral_site','teamA_sval','teamA_off','teamA_scheme','teamB_sval','teamB_def','wind_speed','temperature','precipitation']]
y = df['teamA_pts']

X2 = df[['neutral_site','teamB_sval','teamB_off','teamB_scheme','teamA_sval','teamA_def','wind_speed','temperature','precipitation']]
y2 = df['teamB_pts']


Xf = df[['neutral_site','teamA_sval','teamA_off','teamA_scheme','teamA_def','teamB_sval','teamB_off','teamB_scheme','teamB_def','wind_speed','temperature','precipitation']]
yf = df[['teamA_pts','teamB_pts']]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=64)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,random_state=64)
Xf_train, Xf_test, yf_train, yf_test = train_test_split(Xf, yf, test_size=0.1, random_state=42)

In [30]:
#train 3 models. Team A offense vs Team B defense
model = Ridge(alpha = 1, fit_intercept = True)
model.fit(X_train, y_train)
yp = model.predict(X_test)

model2 = Ridge(alpha = 1, fit_intercept = True)
model2.fit(X2_train, y2_train)
yp2 = model2.predict(X2_test)

modelf = Ridge(alpha = 1, fit_intercept = True)
modelf.fit(Xf_train, yf_train)
ypf = modelf.predict(Xf_test)

In [31]:
print(f'Model Score: {model.score(X_test,y_test)}')
print(f'Model Score: {model2.score(X2_test,y2_test)}')

print(f'Model Score: {modelf.score(Xf_test,yf_test)}')

Model Score: 0.7007255743510125
Model Score: 0.7617234980382508
Model Score: 0.7485225299698816


In [32]:
predict_num = 1

a_ovr = 0.7894
a_off = 0.8033
a_def = 0.9260	
a_scheme = 1.283

b_ovr = 0.6964
b_off = 0.8719	
b_def = 0.8719
b_scheme = 0.975

model_A_predict = [0,a_ovr, a_off, a_scheme, b_ovr, b_def, 0, 75, 0]
model_B_predict = [0,b_ovr, b_off, b_scheme, a_ovr, a_def, 0, 75, 0]
modelf_predict = [0, a_ovr, a_off, a_scheme, a_def, b_ovr, b_off, b_def, b_scheme, 0, 75, 0]



In [33]:
fw_mpr = model.predict([ model_A_predict ])[0]
fw_mpr2 = model2.predict([ model_B_predict ])[0]
mprf = modelf.predict([ modelf_predict ])[0]

print(f"Prediction Home: {fw_mpr}")
print(f"Prediction Away: {fw_mpr2}")
print(f"Prediction Full: {mprf[0]} - {mprf[1]}")

Prediction Home: 35.214752842083165
Prediction Away: 13.49703740690876
Prediction Full: 30.564223759053025 - 15.95542879098987
